In [1]:
# 导入需要的包
import pandas as pd
import jieba
from snownlp import SnowNLP
from snownlp import sentiment

In [2]:
# 载入数据
# 此数据为手工标注的100条情感倾向数据
df = pd.read_csv("cs.csv")  # 数据采集时间是：2020.01.06
df.head()  # 查看数据

,Unnamed: 0,content,sentiment
0,0,喜欢那个打黑拳！！,1.0
1,2,其实已经一口气狂看到了第七卷。前半的节奏还是意气风发无往不利，越到后面那种命运悲剧的感受越强...,1.0
2,3,2020年头的几天假期啥事没干，就看这几百万字了。真的要戒掉看书一定要看完的坏毛病。及时止损。,0.0
3,4,适合消遣看看，破梗废话太多……,0.0
4,5,N年前就看了几章看不下去，因为电视剧 重新拾起看了一遍，觉得还是看电视剧吧…范闲nb的一塌糊涂,0.0


In [3]:
# 删除不需要的列
df = df.drop(["Unnamed: 0"], axis=1)

In [4]:
# 正式的建模过程这里才开始
# 评论列表(分词后)
content_cut_list = df["content"]
# 情感列表
sentiment_list = df["sentiment"]

In [5]:
# 生成snownlp中的训练语料
neg = []
pos = []

# 循环判断语料的积极还是消极倾向，分别把语料放入不同的列表
for i in range(100):
    # 人工标注的消极语料
    if sentiment_list[i] == 0:
        neg.append(content_cut_list[i])
    # 人工标注的积极语料
    if sentiment_list[i] == 1:
        pos.append(content_cut_list[i])

In [6]:
# 把积极和消极的语料写入文件，便于后续的snownlp文件训练
with open("neg.txt", "w+", encoding="utf-8") as f:
    f.write("\n".join(neg))
    
with open("pos.txt", "w+", encoding="utf-8") as f:
    f.write("\n".join(pos))

In [7]:
# 训练前使用snownlp原始模型进行测试
s = SnowNLP('其实已经一口气狂看到了第七卷。前半的节奏还是意气风发无往不利，越到后面那种命运悲剧的感受越强...')
s.sentiments  # 输出情感倾向

0.9786651096458229

In [8]:
# 根据语料训练snownlp模型
sentiment.train('./neg.txt', './pos.txt')
sentiment.save('new.marshal')
# 训练好模型之后需要找到snownlp的目录替换默认的模型
# 根据安装路径的不同：....../$python环境目录/lib/python3.6/site-packages/snownlp/seg/__init__.py
# 修改第九行data_path的值为模型的路径 ....../sentiment.marshal.3

In [9]:
# 除了替换模型的方法，使用下面的load方法也可以加载新的模型
from snownlp import sentiment
sentiment.classifier.load('./new.marshal')
# 情感分析
s = SnowNLP('其实已经一口气狂看到了第七卷。前半的节奏还是意气风发无往不利，越到后面那种命运悲剧的感受越强...')
s.sentiments  # 输出情感倾向
# 可以看到替换模型之后，针对评论书评的情感分析的倾向有了提高

0.9997358262438434